In [1]:
!pip install open3d

## Make sure that the DEVICE that's printed out is 'cuda'. This means that Pytorch is able to use your GPU.

In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import torch
from matplotlib import ticker

np.random.seed(0)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
torch.cuda.empty_cache()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
cuda


## Definition of Gaussian Process Class. Read the comments below for more information about Pytorch.

In [ ]:
### -------------------------------- ###
# Question 2a
class GaussianProcess():
    def __init__(self, X1, Y1, kernel_func=None, noise=1e-2):
        # X1: (N x 3) inputs of training data
        # Y1: (N x 1) outputs of training data
        # kernel_func: (function) a function defining your kernel. It should take the form f(X1, X2) = K, where K is N x N if X1, X2 are N x k.
        # where k is the number of feature dimensions

        self.noise = noise
        self.X1 = X1
        self.Y1 = Y1

        self.kernel_func = kernel_func

        self.compute_training_covariance()

        self.noise_term = self.noise * torch.eye(self.X1.shape[0], device=DEVICE)

    def compute_training_covariance(self):
        # Computes the training covariance matrix Σ11(X, X) using self.kernel_func and your input data self.X1

        ### STUDENT CODE BEGINS ###
        self.Σ11 = ...
        self.Σ11 = self.Σ11 + self.noise_term
        ### STUDENT CODE ENDS ###

    def compute_posterior(self, X):
        # X: (N x k) set of inputs used to predict
        # μ2: (N x 1) GP means at inputs X
        # Σ2: (N x N) GP means at inputs X

        ### STUDENT CODE BEGINS ###
        Σ22 = ...
        Σ12 = ...

        # We've given you this function to speed up inference. There are two things happening here.
        # Instead of solving an explicit inverse, we are solving a system of linear equations
        # self.Σ11 @ X= Σ12, such that X.T = ( Σ21 @ inv(self.Σ11) ). We are using a
        # Cholesky decomposition because Σ11 is PD, so the solver that solves the system of
        # linear equations can save on compute since the matrices have structure to them.
        # You shouldn't need to tweak these next two lines.
        L = torch.linalg.cholesky_ex(self.Σ11)[0]
        Σ_intermediate = torch.cholesky_solve(Σ12, L).T

        # Compute posterior mean
        μ = ...

        # Compute the posterior covariance
        Σ = ...

        ### STUDENT CODE ENDS ###

        return μ, Σ  # posterior mean, covariance

###### KERNELS ######

# Question 2c
def radial_basis(X1, X2, sig=1., l=0.1):
    # Implement the radial basis kernel, given two data matrices X1 and X2
    # X1: Input data (N x k)
    # X2: Input data (M x k)
    # returns K (N x M) matrix

    ### STUDENT CODE BEGINS ###
    N, k = X1.shape
    M, k = X2.shape

    # Suggestions for those unfamiliar with Python / Numpy / Pytorch.
    # Step 1: For every row in X1 and every row in X2 (N x M combinations in total),
    # you want to compute some sort of distance. You can easily do this through array broadcasting.
    # If you convert X1 into a tensor (N x 1 x k) and X2 into a tensor (1 x M x k), their difference
    # will be a (N x M x k) tensor, where the (i,j) element is a k-dimensional vector that represents the
    # difference between the i-th row of X1 and the j-th row of X2. You can use the built-in methods of the tensors:
    # X.reshape(dim0, dim1, dim2).

    # You may find the following functions useful:
    # torch.linalg.norm( ... , dim=-1) -> This takes the norm in the last dimension of a tensor.
    # torch.exp( ... ) -> This is the same as the np equivalent. Takes the element-wise exponent.
    K = ...
    ### STUDENT CODE ENDS ###

    return K


## Read and downsample the point clouds. The relevant files you can put in the Files tab (below Secrets).

In [ ]:
# Load terrain
terrain = o3d.io.read_point_cloud('terrain.ply')
terrain_noisy = o3d.io.read_point_cloud('terrain_noisy.ply')

# Downsample
terrain_down = terrain.uniform_down_sample(100)
terrain_noisy_down = terrain_noisy.uniform_down_sample(100)

## Convert the point clouds into tensors for Pytorch, and create the dataset.

In [ ]:
terrain_down_tensor = torch.tensor(terrain_down.points, device=DEVICE)
terrain_noisy_down_tensor = torch.tensor(terrain_noisy_down.points, device=DEVICE)

X_test = terrain_down_tensor[:,0:2]
Y_test = terrain_down_tensor[:,2]

X_train = terrain_noisy_down_tensor[:,0:2]
Y_train = terrain_noisy_down_tensor[:,2]

## Instantiate the Gaussian Process Class

In [ ]:
gp = GaussianProcess(X_train, Y_train, kernel_func=radial_basis)

## Compute the Posterior on the Training Data

In [ ]:
mu_train, Sigma_train = gp.compute_posterior(X_train)

mu_train = mu_train.cpu().numpy()
Sigma_train = Sigma_train.cpu().numpy()
Y_train = Y_train.cpu().numpy()

## Compute Mean Squared Error on Training

In [ ]:
### STUDENT CODE BEGINS ###
mse_train = ...
### STUDENT CODE ENDS

## Compute the Posterior on the Test Data

In [ ]:
mu_test, Sigma_test = gp.compute_posterior(X_test)
mu_test = mu_test.cpu().numpy()
Sigma_test = Sigma_test.cpu().numpy()

## Compute Mean Squared Error on Test

In [ ]:
### STUDENT CODE BEGINS ###
mse_test = ...
### STUDENT CODE ENDS ###

## Plotting

In [ ]:
### STUDENT CODE BEGINS ###
Y_test = Y_test.cpu().numpy()
X_test = X_test.cpu().numpy()

mean = ...
std = ...
error = ...

### STUDENT CODE ENDS ###


# Use the built-in colormaps and normalization
# Also include the colorbars
fig, ax = plt.subplots(4, figsize=(15, 15), sharex=True)

# Plot the values with a colorbar
vals_to_plot = [Y_test, mean, std, error]
names_to_plot = ['Ground Truth', 'Mean', 'Standard Deviation', 'Difference']

for i, (vals, name) in enumerate(zip(vals_to_plot, names_to_plot)):
    im = ax[i].scatter(X_test[:, 0], X_test[:, 1], c=vals, cmap='viridis')
    ax[i].set_aspect('equal', adjustable='box')

    # Add a y-label (x is shared)
    ax[i].set_ylabel('y')

    # Add a colorbar
    cb = plt.colorbar(im, ax=ax[i], aspect=5)

    # Add more ticks to the colorbar
    tick_locator = ticker.MaxNLocator(nbins=7)
    cb.locator = tick_locator
    cb.update_ticks()

    # Label the colorbar
    cb.set_label(name)

# Add an x-label
ax[-1].set_xlabel('x')